In [54]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import textblob as TextBlob
from bs4 import BeautifulSoup

In [55]:
data = pd.read_excel("tweet1.xlsx")
#data

In [56]:
data.tweet_text.fillna("", inplace=True)
data.tweet_text.dtype

dtype('O')

In [57]:
data['word_count'] = data['tweet_text'].apply(lambda x: len(str(x).split(" ")))
#data.head(10)

In [58]:
import string
data['no_punctuation'] = data['tweet_text'].str.replace('[^\w\s]','')
data.head()

,Unnamed: 0,got_criteria,tweet_text,date,hashtags,link,word_count,no_punctuation
0,0,<GetOldTweets3.models.Tweet.Tweet object at 0x...,Also he’s an unread malignant narcissist who’s...,2020-03-30,NaN,https://twitter.com/SuttleBuddy/status/1244775...,44,Also hes an unread malignant narcissist whos f...
1,1,<GetOldTweets3.models.Tweet.Tweet object at 0x...,Just swap the COVID graphs with the stock mark...,2020-03-30,NaN,https://twitter.com/NotUrplePingo/status/12447...,13,Just swap the COVID graphs with the stock mark...
2,2,<GetOldTweets3.models.Tweet.Tweet object at 0x...,so i think i might’ve waited to long to buy a ...,2020-03-30,NaN,https://twitter.com/Skoog/status/1244763879156...,42,so i think i mightve waited to long to buy a S...
3,3,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"""The stock market is a beast like no other: in...",2020-03-30,#stocks #investing #wjoquotes,https://twitter.com/williamoneilco/status/1244...,36,The stock market is a beast like no other indi...
4,4,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"""Liquidity shall never freeze again"" ICE9 -> S...",2020-03-30,NaN,https://twitter.com/Paisan26849860/status/1244...,26,Liquidity shall never freeze again ICE9 Stock...


In [59]:
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"he’s":"he is","isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not", "might’ve": "might have","y’all":"you all","we're":"we are","you’re":"you "}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def clean_tweet(text):
    try:
        soup = BeautifulSoup(text,'lxml')
        souped = soup.get_text()
    except:
        souped = ""
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

result = []

for tweet in data.tweet_text:
    result.append(clean_tweet(tweet))

data['cleaned_text'] = result

data.head(20)

,Unnamed: 0,got_criteria,tweet_text,date,hashtags,link,word_count,no_punctuation,cleaned_text
0,0,<GetOldTweets3.models.Tweet.Tweet object at 0x...,Also he’s an unread malignant narcissist who’s...,2020-03-30,NaN,https://twitter.com/SuttleBuddy/status/1244775...,44,Also hes an unread malignant narcissist whos f...,also he is an unread malignant narcissist who ...
1,1,<GetOldTweets3.models.Tweet.Tweet object at 0x...,Just swap the COVID graphs with the stock mark...,2020-03-30,NaN,https://twitter.com/NotUrplePingo/status/12447...,13,Just swap the COVID graphs with the stock mark...,just swap the covid graphs with the stock mark...
2,2,<GetOldTweets3.models.Tweet.Tweet object at 0x...,so i think i might’ve waited to long to buy a ...,2020-03-30,NaN,https://twitter.com/Skoog/status/1244763879156...,42,so i think i mightve waited to long to buy a S...,so think might have waited to long to buy swit...
3,3,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"""The stock market is a beast like no other: in...",2020-03-30,#stocks #investing #wjoquotes,https://twitter.com/williamoneilco/status/1244...,36,The stock market is a beast like no other indi...,the stock market is beast like no other indiff...
4,4,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"""Liquidity shall never freeze again"" ICE9 -> S...",2020-03-30,NaN,https://twitter.com/Paisan26849860/status/1244...,26,Liquidity shall never freeze again ICE9 Stock...,liquidity shall never freeze again ice stockma...
5,5,<GetOldTweets3.models.Tweet.Tweet object at 0x...,I’m feeling optimistic that this will end shor...,2020-03-30,NaN,https://twitter.com/InnesKay/status/1244753796...,49,Im feeling optimistic that this will end short...,feeling optimistic that this will end short of...
6,6,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"The pile of scandals remains from before, buri...",2020-03-30,#coronavirus,https://twitter.com/B52Malmet/status/124475179...,48,The pile of scandals remains from before burie...,the pile of scandals remains from before burie...
7,7,<GetOldTweets3.models.Tweet.Tweet object at 0x...,OAN reporter Chanel Rion is a complete kook. I...,2020-03-30,NaN,https://twitter.com/hesanihilist/status/124474...,47,OAN reporter Chanel Rion is a complete kook In...,oan reporter chanel rion is complete kook in h...
8,8,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"Man who called Dow 20,000 at end of 2015 says ...",2020-03-30,NaN,https://twitter.com/MarketWatch/status/1244743...,23,Man who called Dow 20000 at end of 2015 says t...,man who called dow at end of says these are th...
9,9,<GetOldTweets3.models.Tweet.Tweet object at 0x...,The #stockmarket is detached from economic rea...,2020-03-30,#stockmarket,https://twitter.com/SchiffGold/status/12447436...,8,The stockmarket is detached from economic real...,the stockmarket is detached from economic reality


In [31]:
data.to_excel("final_tweet_data.xlsx")
